In [1]:
import pandas as pd
df = pd.read_parquet('df.parquet')

In [3]:
horizon_len = 7
patch_len = 32
patch_num = 20
batch_size = 32
hidden_size = 128
channels = ['Close', 'High','Low',	'Open',	'Volume']

In [ ]:
from deepdow.utils import raw_to_Xy


n_timesteps = len(df)  # 20
n_channels = len(channels)  # 2
#n_assets = len(df.columns.levels[0])  # 2

lookback, gap, horizon = patch_len*patch_num, 2, horizon_len

X, timestamps, y, asset_names, indicators = raw_to_Xy(df,
                                                      lookback=lookback,
                                                      gap=gap,
                                                      freq="B",
                                                      horizon=horizon,
                                                      included_indicators=channels)

n_samples =  n_timesteps - lookback - horizon - gap + 1  # 10
n_assets = len(asset_names)


In [5]:
X.shape

(4309, 5, 640, 30)

In [6]:
timestamps

DatetimeIndex(['2008-09-08', '2008-09-09', '2008-09-10', '2008-09-11',
               '2008-09-12', '2008-09-15', '2008-09-16', '2008-09-17',
               '2008-09-18', '2008-09-19',
               ...
               '2025-02-28', '2025-03-03', '2025-03-04', '2025-03-05',
               '2025-03-06', '2025-03-07', '2025-03-10', '2025-03-11',
               '2025-03-12', '2025-03-13'],
              dtype='datetime64[ns]', length=4309, freq='B')

In [9]:
split_ix = int(n_samples * 0.975)
indices_train = list(range(split_ix))
indices_test = list(range(split_ix + horizon, n_samples))

print('Train range: {}:{}\nTest range: {}:{}'.format(indices_train[0], indices_train[-1],
                                                     indices_test[0], indices_test[-1]))

Train range: 0:4028
Test range: 4036:4132


In [10]:
means, stds = prepare_standard_scaler(X, indices=indices_train)
print('mean: {}, std: {}'.format(means, stds))

mean: [ 0.00043403  0.0004326   0.00043554  0.00043343 -0.00022286], std: [0.01764646 0.01503178 0.01667324 0.01733977 0.3423143 ]


In [11]:
from deepdow.data import InRAMDataset, RigidDataLoader, prepare_standard_scaler, Scale
dataset = InRAMDataset(X, y, transform=Scale(means, stds))

In [13]:
dataloader_train = RigidDataLoader(dataset,
                                   indices=indices_train,
                                   batch_size=batch_size)

dataloader_test = RigidDataLoader(dataset,
                                  indices=indices_test,
                                  batch_size=batch_size)

In [14]:
dataloader_train.next()

AttributeError: 'RigidDataLoader' object has no attribute 'next'